## CPRD Cohort Criteria Examples Workbook

This workbook was created to replicate the example criteria given in CPRD Aurum FAQs v2.4 (see their [website](https://www.cprd.com/primary-care-data-public-health-research)) under the section *"How will I know if the CPRD Aurum data are suitable for my research needs?".* When we include text within quotation marks in the sections below, we are quoting from CPRD Aurum FAQs v2.4. 

We are using these examples to increase our understanding of the tables, how they all relate to eachother and how to write queries for example criteria. These types of queries would allow a research team to filter the CPRD data, to create a sample cohort that matches their research questions e.g. select patients within a certain age range and on a specific medication. 

----
Preliminary setup code:

In [ ]:
# NOTEBOOK SET UP (1) - ask for credentials and db info from user
import getpass
my_username = input('Your username: ')
my_password = getpass.getpass(prompt='Your password: ', stream=None)
this_host = input('Host name: ')
this_db = input('Database name: ')

# NOTEBOOK SET UP (2) - load Jupyter magic functions & connect to db (assumes db & tables already created)
%load_ext sql
%sql postgresql+psycopg2://{my_username}:{my_password}@{this_host}/{this_db}

Below we've written queries for each example criterion step-by-step, hopefully making the complete query easier to understand.

#### Part 1. Counts based on one criterion:

*"The total number of patients in CPRD GOLD or CPRD Aurum with a first ever prescription for
metformin recorded during 01/01/2004 - 31/12/2015, stratified by calendar year"*

a. Find all patients on metformin prescription

In [ ]:
%%sql
-- FILTERING all terms relating to metformin from product dictionary into a new table called A_MetforminProdID
DROP TABLE IF EXISTS  A_MetforminProdID;
SELECT * INTO A_MetforminProdID FROM ProductDictionary 
WHERE (
          UPPER(DrugSubstanceName) LIKE '%METFORMIN%' -- input terms to include here
       );
       
-- Drug issue table contains data relating to all drug prescriptions issued by the GP, 
-- so joining this table to the filtered product dictionary, we'll get all metformin related prescriptions
SELECT DI.*, '<----->' AS BOUNDARY, MP.* FROM DrugIssue DI
INNER JOIN A_MetforminProdID MP ON MP.prodcodeID = DI.prodcodeID
LIMIT 2;

b. Now we have a way of finding all patients with metformin prescriptions issued, we want to find the very first metformin prescription issued, grouped by patient: 


In [ ]:
%%sql
-- Grouping metformin prescription for each patient
SELECT patid, issuedate FROM DrugIssue DI
INNER JOIN A_MetforminProdID MP ON MP.prodcodeID = DI.prodcodeID
GROUP BY patid, issuedate
ORDER BY issuedate ASC
LIMIT 2;

In [ ]:
%%sql
-- Insert above grouping into a new table called A_MetforminPatients:
DROP TABLE IF EXISTS A_MetforminPatients;
SELECT patid, issuedate INTO A_MetforminPatients 
FROM DrugIssue DI
INNER JOIN A_MetforminProdID MP ON MP.prodcodeID = DI.prodcodeID
--WHERE patid = 'XXXXXXXX' -- to look at single patient sample
GROUP BY patid, issuedate
ORDER BY issuedate ASC

-- Ordering to find first metformin prescription for each patient
;WITH added_row_number AS (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY patid ORDER BY issuedate ASC) AS row_number
  FROM A_MetforminPatients
)
SELECT
  *
FROM added_row_number
--WHERE row_number = 1 --AND patid = 'XXXXXXXX' -- single patient sample
LIMIT 20;

c. We have the first metformin prescriptions issued for each patient, so now we want to filter out the those within the date interval given: 01/01/2004 - 31/12/2015 


In [ ]:
%%sql
WITH added_row_number AS (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY patid ORDER BY issuedate ASC) AS row_number
  FROM A_MetforminPatients
)
SELECT
  *
FROM added_row_number
WHERE row_number = 1 --AND patid = 'XXXXXXXXXX'
AND issuedate BETWEEN '2004-01-01' AND '2015-12-31' 
LIMIT 3;

d. The last condition is to stratify by calendar year, which we'll do below:


In [ ]:
%%sql
-- 4. Group by calendar year
WITH added_row_number AS (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY patid ORDER BY issuedate ASC) AS row_number
  FROM A_MetforminPatients
)
SELECT
  DATE_PART('YEAR',issuedate), count(patid)--, issuedate
FROM added_row_number
WHERE row_number = 1 --AND patid = 'XXXXXXXXXX'
AND issuedate BETWEEN '2004-01-01' AND '2015-12-31' 
GROUP BY DATE_PART('YEAR',issuedate)--,patid, issuedate
LIMIT 8;


The above makes use of the product dictionary, and a sample string ('metformin') to filter out relevant drugs. You may however already have a list of drugs that you'd like to load in directly and use to filter out patients, for which we'll have an example below.

----

#### Part 2. Counts based on two criteria:

<i>

1) "Total number of patients with a medical diagnosis of Type 2 diabetes mellitus recorded in CPRD GOLD or HES APC 
on or before 31/12/2005" 
OR
2) "Prescriptions for anti-diabetic medication in CPRD GOLD (note - provided in one code list) 
on or before 31/12/2005. Patients must have at least 12 months of prior registration before their earliest event date."</i>

Rephrased criteria: 

1) Total number of patients with a medical diagnosis of Type 2 diabetes mellitus recorded in CPRD Aurum on or before 31/12/2005 
OR
2) Prescriptions for anti-diabetic medication in CPRD Aurum (note - provided in one code list) on or before 31/12/2005. 
Patients must have at least 12 months of prior registration before their earliest event date.

We'll split up both criteria (1 & 2) and start with the first one, in segments, below:



1a. Load in pre-existing code list for Type2Diabetes, CPRD Aurum, from OPTIMAL's THINK repo (or any other of your own choice)

<i>link here: https://github.com/THINKINGGroup/phenotypes/tree/main/Medical%20conditions/Type2Diabetes_11_3_21_birm_cam</i>


In [ ]:
# Specify file path for your downloaded code list file (for example: User/Docs/codelists/type2diabetes.csv)
code_list_dir = input('Enter path for type 2 diabetes code list file')

In [ ]:
%%sql
-- Create a new table to load list into (make sure this matches the headers in your code list file!)
DROP TABLE IF EXISTS A_Type2Diabetes_CodeLists;
CREATE TABLE A_Type2Diabetes_CodeLists
(MEDICAL_CODE_ID TEXT,DESCRIPTION TEXT,READ_CODE TEXT,SNOMED_CT_CODE TEXT,DATASOURCE TEXT);

In [ ]:
%%sql
-- Loading in code list from directory specified above
DO $$ 
DECLARE
    filepath TEXT := :code_list_dir;
BEGIN
    EXECUTE 'COPY A_Type2Diabetes_CodeLists (MEDICAL_CODE_ID,DESCRIPTION,READ_CODE,SNOMED_CT_CODE,DATASOURCE) FROM ''' || filepath || ''' WITH (FORMAT ''csv'', DELIMITER '','', HEADER, QUOTE ''"'');';
END $$;

In [ ]:
%%sql
-- View table contents 
SELECT * FROM A_Type2Diabetes_CodeLists LIMIT 2;

In [ ]:
%%sql
-- Use code list to filter medcodeids in medical dictionary for Type2Diabetes
SELECT * FROM medicaldictionary
WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists)
LIMIT 2;

1b. Find all type 2 diabetes related observations

In [ ]:
%%sql
-- View patients with type2db medcodeid in observation table
SELECT * FROM observation 
WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists)
LIMIT 2;

In [ ]:
%%sql
--Total observations related to type2db
(SELECT 'Total obs related to typ2db' AS Totals, count(*) FROM observation
WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists))
UNION
--  Total distinct patients with observations related to/medical diagnosis of Type 2 diabetes mellitus 
(SELECT 'Total distinct patients with obs related to typ2db', count(DISTINCT patid) FROM observation
WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists));

1c. Medical diagnosis (inferred by a medcodeid occurence) recorded on or before 31/12/2005:

<i> Note to check assumptions of methodology below: we might want to join the problem table and consider the parent problem label - we did it manually for now. </i>


In [ ]:
%%sql 
-- Find all type2d observations grouped (least to most recent, ascending) for each patient
SELECT patid,obsdate,medcodeid,t.description FROM observation o 
INNER JOIN A_Type2Diabetes_CodeLists t ON t.medical_code_id = o.medcodeid 
WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists)
GROUP BY patid,obsdate,medcodeid, t.description
ORDER BY patid,obsdate ASC
LIMIT 5;

In [ ]:
%%sql 
-- Find earliest observation and add constraint for observation date
WITH patid_ranked AS (
SELECT *, 
ROW_NUMBER() OVER(PARTITION BY patid ORDER BY obsdate ASC) AS row_number
FROM observation WHERE medcodeid IN (SELECT medical_code_id FROM A_Type2Diabetes_CodeLists)
)
SELECT * FROM patid_ranked
WHERE row_number = 1 -- most recent observation when ordered by obsdate ascending
AND obsdate <= '2005-12-31' -- add date constraint for the first observation 
LIMIT 3; 


Now we'll look at the second criteria:

2. Prescriptions for anti-diabetic medication in CPRD Aurum (note - provided in one code list) on or before 31/12/2005.  
Patients must have at least 12 months of prior registration before their earliest event date.

<i>Edit: date modified from 31/12/2005 to 31/12/2015 as this was a more suitable range for the synthetic Aurum dataset. If you are running this notebook on the real Aurum dataset, try the original date threshold.</i> 

2a. Load in multiple code lists from OPTIMAL's THINK repo (or any resource of your own preference) that relate to anti-diabetic medications.

<i>links here: 
- https://github.com/THINKINGGroup/phenotypes/blob/main/Drug%20Codes
- https://phenotypes.healthdatagateway.org/phenotypes/</i>


In [ ]:
%%sql 
-- Create a new filename table, as there are multiple antidiabetic drug files
DROP TABLE IF EXISTS A_AntiDiabeticDrug_CodeList_Name;
CREATE TABLE A_AntiDiabeticDrug_CodeList_Name(FILEID INT GENERATED BY DEFAULT AS IDENTITY, DRUGFILENAME TEXT);
SELECT * FROM A_AntiDiabeticDrug_CodeList_Name;

In [ ]:
import os
## Change directory here to the folder containing the code lists:
directory_path = input('Specify directory for folder containing the diabetic medication code lists')

# Iterate over all the code lists in the specified directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Insert filenames into the table
        %sql INSERT INTO A_AntiDiabeticDrug_CodeList_Name (DRUGFILENAME) VALUES (:filename)


In [ ]:
%%sql
-- Create a new antidiabetic medication code list table (note this should match your code list file headers):
DROP TABLE IF EXISTS A_AntiDiabeticDrug_CodeLists;
CREATE TABLE A_AntiDiabeticDrug_CodeLists
(DRUG_CODE_ID TEXT,DESCRIPTION TEXT,BNF1 TEXT,BNF2 TEXT,BNF3 TEXT,ATC TEXT,DATABASE TEXT);

Create function to load in data below, which we will then use to loop over multiple code lists

<i>*You will need to change the values according to your own code list file contents </i>


In [ ]:
%%sql
-- Function to load code lists into table in a LOOP
CREATE OR REPLACE FUNCTION A_loaddata(filepathname TEXT)
  RETURNS void
  LANGUAGE plpgsql AS
$func$
BEGIN
   EXECUTE format ('
   COPY A_AntiDiabeticDrug_CodeLists(DRUG_CODE_ID,DESCRIPTION,BNF1,BNF2,BNF3,ATC,DATABASE)  -- more columns
   FROM %L (FORMAT CSV, HEADER)'  -- modern syntax
           -- WITH CSV HEADER'    -- tolerated legacy syntax
   , $1);  -- pass 1st function parameter (filepathname) to format() 
END
$func$;

Now, loop over drug list filenames and apply loaddata() function:


In [ ]:
%%sql 
-- SELECT * FROM A_AntiDiabeticDrug_CodeList_Name; -- contains filenames
-- SELECT * FROM A_AntiDiabeticDrug_CodeLists; -- table to insert into

--Loop loaddata() function over drug list filenames to insert into table:
do $$
DECLARE fileName VARCHAR;
BEGIN
FOR fileName IN 
SELECT drugfilename FROM A_AntiDiabeticDrug_CodeList_Name
LOOP
    PERFORM A_loaddata(
                    :directory_path || '/' || fileName
                    );
END LOOP;
END;
$$;

In [ ]:
%%sql 
--Check table for loaded data:
SELECT * FROM A_AntiDiabeticDrug_CodeLists
LIMIT 2;

2b. Now we want to use the antidiabetic code lists table to filter out the relevant drugs from dictionary (or we can use code list table directly, since it already has cprd aurum identifiers if it's a complete match)


In [ ]:
%%sql 
-- a. Use code list to filter prodcodeids in product dictionary for anti-diabetic medication
SELECT * FROM productdictionary 
WHERE PRODCODEID IN (SELECT drug_code_id FROM A_AntiDiabeticDrug_CodeLists)
LIMIT 2;

In [ ]:
%%sql 
-- You can also check if there's a count match between all the codes in your code list vs. codes in the product dictionary table as a sanity check
(SELECT COUNT(DISTINCT drug_code_id) FROM A_AntiDiabeticDrug_CodeLists) --
UNION ALL
(SELECT count(*) FROM productdictionary 
WHERE PRODCODEID IN (SELECT drug_code_id FROM A_AntiDiabeticDrug_CodeLists)
);

2c. Find patient observations relating to antidiabetic drugs

In [ ]:
%%sql
SELECT DI.*, ADC.* FROM drugissue DI 
INNER JOIN A_AntiDiabeticDrug_CodeLists ADC ON DI.prodcodeid = ADC.drug_code_id
WHERE DI.issuedate <= '2015-12-31'
ORDER BY enterdate ASC
LIMIT 2;

-- Total patient prescriptions with antidiabetic drugs
SELECT COUNT(DISTINCT patid) FROM drugissue DI 
INNER JOIN A_AntiDiabeticDrug_CodeLists ADC ON DI.prodcodeid = ADC.drug_code_id
WHERE DI.issuedate <= '2015-12-31'
LIMIT 2;

2d. Patients must have at least 12 months of prior registration before their earliest event date (earliest prescription).

So below we have a table with earliest prescription issue date for each patient on antidiabetic drugs, where patients also have AT LEAST a one year interval between first issue of metformin and registration date at GP clinic 


In [ ]:
%%sql 
-- Table with earliest prescription issue date 
-- for each patient on antidiabetic drugs
WITH patid_ranked AS (
        SELECT  DI.*, 
                ROW_NUMBER() OVER(PARTITION BY patid ORDER BY issuedate ASC) AS row_number,
                ADC.*
        FROM drugissue DI 
        INNER JOIN A_AntiDiabeticDrug_CodeLists ADC ON DI.prodcodeid = ADC.drug_code_id
        WHERE DI.issuedate <= '2015-12-31'
                    )
SELECT P.regstartdate,R.issuedate,R.issuedate - P.regstartdate AS DIFF, R.issuedate - interval '1 year' AS IssueDateMinus1Year,R.* FROM patid_ranked R
    INNER JOIN Patient P ON P.patid = R.patid
    WHERE row_number = 1
    AND P.regstartdate <= (R.issuedate - interval '1 year')
    ORDER BY DIFF ASC
    LIMIT 3;

------

#### Part 3. Counts based on three criteria:
1) The total number of patients in CPRD Aurum with an incident medical
diagnosis of Type 2 diabetes mellitus recorded in during 01/01/2004 - 31/12/2015 OR
2) Incident prescription of anti-diabetic medication (note-provided in one code list) documented during
01/01/2004 - 31/12/2015 AND
3) Have a test record for HbA1c recorded in CPRD Aurum (note - test value not assessed)

First and second criteria have already been previously covered; here we focus on the third criteria only.




3a. Find all terms related to '%HbA1%' and insert into a new table called A_hba1_medcodes


In [ ]:
%%sql 
--SELECT * FROM Observation LIMIT 10;
--SELECT * FROM obstype ORDER BY 1; -- lookup table
DROP TABLE IF EXISTS A_hba1_medcodes;
SELECT * INTO A_hba1_medcodes FROM medicaldictionary
WHERE term LIKE '%HbA1%';
--View table values:
SELECT * FROM A_hba1_medcodes LIMIT 2;


3b. Finding all patients with HbA1C related observations, where the observation type is a 'value' and the value is populated (not a NULL value)

<i>*Note, assumption made below, that 'value' obstype gives the test record</i>


In [ ]:
%%sql

-- if we want to see all the distinct observation types 
SELECT DISTINCT obs.obstypeid FROM Observation obs
INNER JOIN obstype ot ON ot.obstypeid = obs.obstypeid
WHERE obs.medcodeid IN (SELECT medcodeid FROM A_hba1_medcodes)
LIMIT 2;

--checking what different ids correspond to
SELECT * FROM numunit;

--Joining tables to see observations for patients where 
--type of observation is a 'value' type, not null, 
-- and related to 'hba1c' term in dictionary 
SELECT obs.*, ot.*, md.* FROM Observation obs
INNER JOIN obstype ot ON ot.obstypeid = obs.obstypeid
INNER JOIN medicaldictionary md ON md.medcodeid = obs.medcodeid
WHERE obs.medcodeid IN (SELECT medcodeid FROM A_hba1_medcodes)
AND ot.obstypeid = 10 -- value
AND obs.value IS NOT NULL
LIMIT 2;

3c. We can also now count the distinct number of patients:

In [ ]:
%%sql
--count distinct patients:
SELECT COUNT(DISTINCT obs.patid) FROM Observation obs
INNER JOIN obstype ot ON ot.obstypeid = obs.obstypeid
INNER JOIN medicaldictionary md ON md.medcodeid = obs.medcodeid
WHERE obs.medcodeid IN (SELECT medcodeid FROM A_hba1_medcodes)
AND ot.obstypeid = 10 -- value
AND obs.value IS NOT NULL
LIMIT 2;


------

#### Drop tables created in this notebook:


In [ ]:
%%sql
DROP TABLE IF EXISTS a_antidiabeticdrug_codelist_name;
DROP TABLE IF EXISTS a_antidiabeticdrug_codelists;
DROP TABLE IF EXISTS a_metforminpatients;
DROP TABLE IF EXISTS a_metforminprodid;
DROP TABLE IF EXISTS a_type2diabetes_codelists;
DROP TABLE IF EXISTS a_hba1_medcodes;
DROP FUNCTION IF EXISTS a_loaddata;